In [1]:
import os
import zipfile
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers, datasets

from collections import Counter

import openpifpaf
import cv2
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import os
from glob import glob
try:
    from PIL import Image
except ImportError:
    import Image
from time import time
from tqdm import tqdm, tqdm_notebook
import json

import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv3D, MaxPooling3D, Dropout, BatchNormalization, LSTM, ConvLSTM2D, TimeDistributed, Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split, LeaveOneOut
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from xgboost.sklearn import XGBClassifier
import pickle 
import warnings
warnings.filterwarnings("ignore")

In [2]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [3]:
skeletons_action_folders = glob('./datasets/Weizmann/skeletons_croppeds/*')

In [4]:
skeletons_action_folders[0].split('\\')[1]

'bend'

In [5]:
dict_data = {'label': [], 'average_image': []}
for skeleton_folder in tqdm_notebook(skeletons_action_folders):
    
    current_action = skeleton_folder.split('\\')[1]
    
    actors_folders = glob(skeleton_folder.replace('\\', '/')+'/*')
    
    for actor_folder in actors_folders:
        
        dict_data['label'].append(current_action)
        
        images = glob(actor_folder.replace('\\', '/')+'/*')
        
        w,h=Image.open(images[0]).size
        N=len(images)

        # Create a numpy array of floats to store the average (assume RGB images)
        arr=np.zeros((h,w,3),np.float)

        # Build up average pixel intensities, casting each image as an array of floats
        for im in images:
            imarr=np.array(Image.open(im),dtype=np.float)
            arr=arr+imarr/N

        # Round values in array and cast as 8-bit integer
        arr=np.array(np.round(arr),dtype=np.uint8)
        
        dict_data['average_image'].append(arr)
        # Generate, save and preview final image
        out=Image.fromarray(arr,mode="RGB")
        out.save("Average.png")

  0%|          | 0/10 [00:00<?, ?it/s]

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt



In [6]:
dict_data['average_image'][0].shape

(100, 100, 3)

In [7]:
df = pd.DataFrame(dict_data)

# Logistic Regression

In [27]:
X_train, X_test, y_train, y_test = train_test_split(df['average_image'].values, df['label'].values, test_size=0.3, random_state=42)

In [28]:
X_train = [item.ravel() for item in X_train]

In [29]:
X_test = [item.ravel() for item in X_test]

In [30]:
clf = LogisticRegression()

In [31]:
clf.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [32]:
y_pred = clf.predict(X_test)

In [33]:
y_pred[5]

'jump'

In [34]:
y_test[5]

'jump'

In [35]:
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

        bend       1.00      1.00      1.00         2
        jack       1.00      1.00      1.00         4
        jump       0.60      1.00      0.75         3
       pjump       1.00      1.00      1.00         4
         run       0.60      1.00      0.75         3
        side       0.50      1.00      0.67         1
        skip       1.00      0.50      0.67         2
        walk       1.00      0.33      0.50         6
       wave1       1.00      1.00      1.00         2
       wave2       1.00      1.00      1.00         1

    accuracy                           0.82        28
   macro avg       0.87      0.88      0.83        28
weighted avg       0.90      0.82      0.80        28

0.8214285714285714


In [8]:
loo = LeaveOneOut()

In [37]:
loo.get_n_splits(df['average_image'].values)

93

In [38]:
y_pred_all = []
y_true_all = []
for train_index, test_index in tqdm_notebook(loo.split(df['average_image'].values)):
#     print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = df['average_image'].values[train_index], df['average_image'].values[test_index]
    y_train, y_test = df['label'].values[train_index], df['label'].values[test_index]

    X_train = [item.ravel() for item in X_train]
    X_test = [item.ravel() for item in X_test]
    
    clf = LogisticRegression()
    
    clf.fit(X_train, y_train)
    
    y_pred = clf.predict(X_test)
    
    y_pred_all.append(y_pred[0])
    y_true_all.append(y_test[0])

0it [00:00, ?it/s]

In [39]:
print(classification_report(y_true_all, y_pred_all))

              precision    recall  f1-score   support

        bend       1.00      1.00      1.00         9
        jack       1.00      1.00      1.00         9
        jump       0.70      0.78      0.74         9
       pjump       0.90      1.00      0.95         9
         run       0.80      0.80      0.80        10
        side       1.00      0.89      0.94         9
        skip       0.56      0.50      0.53        10
        walk       0.83      1.00      0.91        10
       wave1       1.00      0.89      0.94         9
       wave2       1.00      0.89      0.94         9

    accuracy                           0.87        93
   macro avg       0.88      0.87      0.87        93
weighted avg       0.87      0.87      0.87        93



In [40]:
print(accuracy_score(y_true_all, y_pred_all))

0.8709677419354839


# CNN

In [63]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(100,100,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [64]:
optimizer = tf.keras.optimizers.Adam()
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [43]:
df['average_image'] = df['average_image'].apply(lambda x: np.asarray(x).astype(np.float32))

In [44]:
X_train, X_test, y_train, y_test = train_test_split(df['average_image'].values, df['label'].values, test_size=0.3, random_state=42)

In [45]:
X_train = X_train / 255.
X_test = X_test / 255.

In [46]:
# X_train = [np.reshape(item, (-1, 160,160, 3)) for item in X_train]

In [47]:
# X_test = [np.reshape(item, (-1, 160,160, 3)) for item in X_test]

In [10]:
dict_map = {}
for i, label in enumerate(df['label'].unique()):
    
    dict_map[label] = i

In [11]:
dict_map

{'bend': 0,
 'jack': 1,
 'jump': 2,
 'pjump': 3,
 'run': 4,
 'side': 5,
 'skip': 6,
 'walk': 7,
 'wave1': 8,
 'wave2': 9}

In [50]:
y_test = np.array([dict_map[y_t] for y_t in y_test])

In [51]:
y_train = np.array([dict_map[y_t] for y_t in y_train])

In [52]:
X_train = [np.asarray(x).astype(np.float32) for x in X_train]

In [53]:
X_test = [np.asarray(x).astype(np.float32) for x in X_test]

In [54]:
X_train = np.array(X_train)

In [55]:
X_test = np.array(X_test)

In [84]:
tf.config.run_functions_eagerly(True)

In [65]:
model.fit(X_train, y_train,
          epochs=50,
          validation_data=(X_test,y_test))

Epoch 1/50
3/3 [==============================] - 1s 112ms/step - loss: 2.2671 - accuracy: 0.2769 - val_loss: 2.1477 - val_accuracy: 0.3571
Epoch 2/50
3/3 [==============================] - 0s 63ms/step - loss: 1.9945 - accuracy: 0.3538 - val_loss: 2.0483 - val_accuracy: 0.2857
Epoch 3/50
3/3 [==============================] - 0s 65ms/step - loss: 1.7550 - accuracy: 0.3692 - val_loss: 1.9469 - val_accuracy: 0.2857
Epoch 4/50
3/3 [==============================] - 0s 66ms/step - loss: 1.4638 - accuracy: 0.6000 - val_loss: 1.6508 - val_accuracy: 0.4286
Epoch 5/50
3/3 [==============================] - 0s 62ms/step - loss: 1.1474 - accuracy: 0.6462 - val_loss: 1.4370 - val_accuracy: 0.5357
Epoch 6/50
3/3 [==============================] - 0s 62ms/step - loss: 0.9219 - accuracy: 0.7538 - val_loss: 1.3753 - val_accuracy: 0.5000
Epoch 7/50
3/3 [==============================] - 0s 62ms/step - loss: 0.8225 - accuracy: 0.7231 - val_loss: 1.1462 - val_accuracy: 0.5357
Epoch 8/50
3/3 [==========

In [66]:
preds = model.predict(X_test)

In [67]:
y_pred = [pred.argmax() for pred in preds]

In [68]:
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         4
           2       0.50      0.67      0.57         3
           3       1.00      1.00      1.00         4
           4       0.60      1.00      0.75         3
           5       1.00      1.00      1.00         1
           6       0.33      0.50      0.40         2
           7       1.00      0.33      0.50         6
           8       1.00      1.00      1.00         2
           9       1.00      1.00      1.00         1

    accuracy                           0.79        28
   macro avg       0.84      0.85      0.82        28
weighted avg       0.86      0.79      0.78        28

0.7857142857142857


In [12]:
y_pred_all = []
y_true_all = []
acertos = 0
for train_index, test_index in tqdm_notebook(loo.split(df['average_image'].values)):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(100,100,3)),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    
    optimizer = tf.keras.optimizers.Adam()
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    
#     print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = df['average_image'].values[train_index], df['average_image'].values[test_index]
    y_train, y_test = df['label'].values[train_index], df['label'].values[test_index]
    
    X_train = X_train / 255.
    X_test = X_test / 255.
    
    y_test = np.array([dict_map[y_t] for y_t in y_test])

    y_train = np.array([dict_map[y_t] for y_t in y_train])

    X_train = [np.asarray(x).astype(np.float32) for x in X_train]

    X_test = [np.asarray(x).astype(np.float32) for x in X_test]

    X_train = np.array(X_train)

    X_test = np.array(X_test)
    
    model.fit(X_train, y_train,
          epochs=70,
          validation_data=(X_test,y_test))
    
    preds = model.predict(X_test)
    
    y_pred = [pred.argmax() for pred in preds]
    
    y_pred_all.append(y_pred[0])
    y_true_all.append(y_test[0])
    
    if y_pred[0] == y_test[0]:
            acertos += 1
    print(str(acertos)+'/'+str(len(y_pred_all)))

0it [00:00, ?it/s]

Epoch 1/70
3/3 [==============================] - 2s 184ms/step - loss: 2.2772 - accuracy: 0.0870 - val_loss: 2.2242 - val_accuracy: 0.0000e+00
Epoch 2/70
3/3 [==============================] - 0s 80ms/step - loss: 1.9535 - accuracy: 0.4891 - val_loss: 2.0181 - val_accuracy: 0.0000e+00
Epoch 3/70
3/3 [==============================] - 0s 80ms/step - loss: 1.5552 - accuracy: 0.5652 - val_loss: 1.5586 - val_accuracy: 1.0000
Epoch 4/70
3/3 [==============================] - 0s 81ms/step - loss: 1.1290 - accuracy: 0.7065 - val_loss: 0.5186 - val_accuracy: 1.0000
Epoch 5/70
3/3 [==============================] - 0s 80ms/step - loss: 0.8297 - accuracy: 0.7826 - val_loss: 0.2883 - val_accuracy: 1.0000
Epoch 6/70
3/3 [==============================] - 0s 79ms/step - loss: 0.5026 - accuracy: 0.9239 - val_loss: 0.3766 - val_accuracy: 1.0000
Epoch 7/70
3/3 [==============================] - 0s 80ms/step - loss: 0.3799 - accuracy: 0.9022 - val_loss: 0.3645 - val_accuracy: 1.0000
Epoch 8/70
3/3 [==

Epoch 44/70
3/3 [==============================] - 0s 80ms/step - loss: 1.9798e-04 - accuracy: 1.0000 - val_loss: 8.8355e-04 - val_accuracy: 1.0000
Epoch 45/70
3/3 [==============================] - 0s 80ms/step - loss: 1.8663e-04 - accuracy: 1.0000 - val_loss: 8.3733e-04 - val_accuracy: 1.0000
Epoch 46/70
3/3 [==============================] - 0s 82ms/step - loss: 1.7607e-04 - accuracy: 1.0000 - val_loss: 8.0732e-04 - val_accuracy: 1.0000
Epoch 47/70
3/3 [==============================] - 0s 81ms/step - loss: 1.6786e-04 - accuracy: 1.0000 - val_loss: 7.7623e-04 - val_accuracy: 1.0000
Epoch 48/70
3/3 [==============================] - 0s 81ms/step - loss: 1.5910e-04 - accuracy: 1.0000 - val_loss: 7.5312e-04 - val_accuracy: 1.0000
Epoch 49/70
3/3 [==============================] - 0s 80ms/step - loss: 1.4990e-04 - accuracy: 1.0000 - val_loss: 7.3763e-04 - val_accuracy: 1.0000
Epoch 50/70
3/3 [==============================] - 0s 80ms/step - loss: 1.4274e-04 - accuracy: 1.0000 - val_loss

3/3 [==============================] - 0s 87ms/step - loss: 0.0090 - accuracy: 1.0000 - val_loss: 0.0469 - val_accuracy: 1.0000
Epoch 17/70
3/3 [==============================] - 0s 83ms/step - loss: 0.0072 - accuracy: 1.0000 - val_loss: 0.0745 - val_accuracy: 1.0000
Epoch 18/70
3/3 [==============================] - 0s 82ms/step - loss: 0.0047 - accuracy: 1.0000 - val_loss: 0.1156 - val_accuracy: 1.0000
Epoch 19/70
3/3 [==============================] - 0s 82ms/step - loss: 0.0028 - accuracy: 1.0000 - val_loss: 0.1670 - val_accuracy: 1.0000
Epoch 20/70
3/3 [==============================] - 0s 82ms/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 0.2236 - val_accuracy: 1.0000
Epoch 21/70
3/3 [==============================] - 0s 85ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 0.2712 - val_accuracy: 1.0000
Epoch 22/70
3/3 [==============================] - 0s 84ms/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.3018 - val_accuracy: 1.0000
Epoch 23/70
3/3 [===============

3/3 [==============================] - 0s 88ms/step - loss: 2.6645e-04 - accuracy: 1.0000 - val_loss: 2.5630e-05 - val_accuracy: 1.0000
Epoch 60/70
3/3 [==============================] - 0s 89ms/step - loss: 2.5866e-04 - accuracy: 1.0000 - val_loss: 2.4914e-05 - val_accuracy: 1.0000
Epoch 61/70
3/3 [==============================] - 0s 86ms/step - loss: 2.5293e-04 - accuracy: 1.0000 - val_loss: 2.3126e-05 - val_accuracy: 1.0000
Epoch 62/70
3/3 [==============================] - 0s 100ms/step - loss: 2.4414e-04 - accuracy: 1.0000 - val_loss: 2.2292e-05 - val_accuracy: 1.0000
Epoch 63/70
3/3 [==============================] - 0s 89ms/step - loss: 2.3847e-04 - accuracy: 1.0000 - val_loss: 2.0504e-05 - val_accuracy: 1.0000
Epoch 64/70
3/3 [==============================] - 0s 86ms/step - loss: 2.3162e-04 - accuracy: 1.0000 - val_loss: 1.9908e-05 - val_accuracy: 1.0000
Epoch 65/70
3/3 [==============================] - 0s 84ms/step - loss: 2.2548e-04 - accuracy: 1.0000 - val_loss: 1.9193e-0

...

3/3 [==============================] - 0s 84ms/step - loss: 3.8971e-04 - accuracy: 1.0000 - val_loss: 0.0056 - val_accuracy: 1.0000
Epoch 49/70
3/3 [==============================] - 0s 89ms/step - loss: 3.8138e-04 - accuracy: 1.0000 - val_loss: 0.0055 - val_accuracy: 1.0000
Epoch 50/70
3/3 [==============================] - 0s 91ms/step - loss: 3.7051e-04 - accuracy: 1.0000 - val_loss: 0.0054 - val_accuracy: 1.0000
Epoch 51/70
3/3 [==============================] - 0s 87ms/step - loss: 3.6025e-04 - accuracy: 1.0000 - val_loss: 0.0053 - val_accuracy: 1.0000
Epoch 52/70
3/3 [==============================] - 0s 87ms/step - loss: 3.4946e-04 - accuracy: 1.0000 - val_loss: 0.0052 - val_accuracy: 1.0000
Epoch 53/70
3/3 [==============================] - 0s 87ms/step - loss: 3.4159e-04 - accuracy: 1.0000 - val_loss: 0.0052 - val_accuracy: 1.0000
Epoch 54/70
3/3 [==============================] - 0s 87ms/step - loss: 3.3485e-04 - accuracy: 1.0000 - val_loss: 0.0051 - val_accuracy: 1.0000
Epoc

Epoch 35/70
3/3 [==============================] - 0s 91ms/step - loss: 5.5743e-04 - accuracy: 1.0000 - val_loss: 8.3446e-07 - val_accuracy: 1.0000
Epoch 36/70
3/3 [==============================] - 0s 89ms/step - loss: 5.2751e-04 - accuracy: 1.0000 - val_loss: 7.1526e-07 - val_accuracy: 1.0000
Epoch 37/70
3/3 [==============================] - 0s 132ms/step - loss: 5.0019e-04 - accuracy: 1.0000 - val_loss: 7.1526e-07 - val_accuracy: 1.0000
Epoch 38/70
3/3 [==============================] - 1s 261ms/step - loss: 4.7522e-04 - accuracy: 1.0000 - val_loss: 5.9605e-07 - val_accuracy: 1.0000
Epoch 39/70
3/3 [==============================] - 1s 221ms/step - loss: 4.5402e-04 - accuracy: 1.0000 - val_loss: 5.9605e-07 - val_accuracy: 1.0000
Epoch 40/70
3/3 [==============================] - 0s 86ms/step - loss: 4.3257e-04 - accuracy: 1.0000 - val_loss: 4.7684e-07 - val_accuracy: 1.0000
Epoch 41/70
3/3 [==============================] - 0s 88ms/step - loss: 4.1275e-04 - accuracy: 1.0000 - val_l

3/3 [==============================] - 0s 87ms/step - loss: 0.3469 - accuracy: 0.9022 - val_loss: 3.8175e-04 - val_accuracy: 1.0000
Epoch 9/70
3/3 [==============================] - 0s 91ms/step - loss: 0.2179 - accuracy: 0.9457 - val_loss: 9.5244e-05 - val_accuracy: 1.0000
Epoch 10/70
3/3 [==============================] - 0s 114ms/step - loss: 0.1881 - accuracy: 0.9674 - val_loss: 5.7219e-05 - val_accuracy: 1.0000
Epoch 11/70
3/3 [==============================] - 1s 264ms/step - loss: 0.0990 - accuracy: 0.9783 - val_loss: 1.4663e-05 - val_accuracy: 1.0000
Epoch 12/70
3/3 [==============================] - 1s 241ms/step - loss: 0.0643 - accuracy: 1.0000 - val_loss: 7.3909e-06 - val_accuracy: 1.0000
Epoch 13/70
3/3 [==============================] - 0s 84ms/step - loss: 0.0398 - accuracy: 1.0000 - val_loss: 3.6955e-06 - val_accuracy: 1.0000
Epoch 14/70
3/3 [==============================] - 0s 87ms/step - loss: 0.0324 - accuracy: 0.9891 - val_loss: 2.3842e-06 - val_accuracy: 1.0000
Ep

Epoch 64/70
3/3 [==============================] - 0s 90ms/step - loss: 2.5322e-04 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 65/70
3/3 [==============================] - 0s 89ms/step - loss: 2.4815e-04 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 66/70
3/3 [==============================] - 0s 142ms/step - loss: 2.4139e-04 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 67/70
3/3 [==============================] - 1s 298ms/step - loss: 2.3766e-04 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 68/70
3/3 [==============================] - 1s 205ms/step - loss: 2.3171e-04 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 69/70
3/3 [==============================] - 0s 84ms/step - loss: 2.2726e-04 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 70/70
3/3 [==============================] - 0s 88ms/step - loss: 2.2260e-04 - accuracy: 1.0000 - val_l

3/3 [==============================] - 0s 90ms/step - loss: 3.5020e-04 - accuracy: 1.0000 - val_loss: 2.0266e-06 - val_accuracy: 1.0000
Epoch 37/70
3/3 [==============================] - 0s 90ms/step - loss: 3.3136e-04 - accuracy: 1.0000 - val_loss: 1.9073e-06 - val_accuracy: 1.0000
Epoch 38/70
3/3 [==============================] - 0s 90ms/step - loss: 3.1976e-04 - accuracy: 1.0000 - val_loss: 1.7881e-06 - val_accuracy: 1.0000
Epoch 39/70
3/3 [==============================] - 0s 90ms/step - loss: 3.0004e-04 - accuracy: 1.0000 - val_loss: 1.7881e-06 - val_accuracy: 1.0000
Epoch 40/70
3/3 [==============================] - 0s 89ms/step - loss: 2.8319e-04 - accuracy: 1.0000 - val_loss: 1.9073e-06 - val_accuracy: 1.0000
Epoch 41/70
3/3 [==============================] - 0s 91ms/step - loss: 2.7086e-04 - accuracy: 1.0000 - val_loss: 1.7881e-06 - val_accuracy: 1.0000
Epoch 42/70
3/3 [==============================] - 0s 90ms/step - loss: 2.5176e-04 - accuracy: 1.0000 - val_loss: 1.5497e-06

In [13]:
print(classification_report(y_true_all, y_pred_all))
print(accuracy_score(y_true_all, y_pred_all))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         9
           1       1.00      1.00      1.00         9
           2       0.70      0.78      0.74         9
           3       0.90      1.00      0.95         9
           4       0.88      0.70      0.78        10
           5       1.00      1.00      1.00         9
           6       0.75      0.60      0.67        10
           7       0.83      1.00      0.91        10
           8       0.89      0.89      0.89         9
           9       1.00      1.00      1.00         9

    accuracy                           0.89        93
   macro avg       0.89      0.90      0.89        93
weighted avg       0.89      0.89      0.89        93

0.8924731182795699


# RNN

In [5]:
def create_LRCN_model():
    '''
    This function will construct the required LRCN model.
    Returns:
        model: It is the required constructed LRCN model.
    '''

    # We will use a Sequential model for model construction.
    model = Sequential()
    
    # Define the Model Architecture.
    ########################################################################################################################
    
    model.add(TimeDistributed(Conv2D(16, (3, 3), padding='same',activation = 'relu'),
                              input_shape = (29, 100, 100, 3)))
    
    model.add(TimeDistributed(MaxPooling2D((4, 4)))) 
    model.add(TimeDistributed(Dropout(0.25)))
    
    model.add(TimeDistributed(Conv2D(32, (3, 3), padding='same',activation = 'relu')))
    model.add(TimeDistributed(MaxPooling2D((4, 4))))
    model.add(TimeDistributed(Dropout(0.25)))
    
    model.add(TimeDistributed(Conv2D(64, (3, 3), padding='same',activation = 'relu')))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    model.add(TimeDistributed(Dropout(0.25)))
    
    model.add(TimeDistributed(Conv2D(64, (3, 3), padding='same',activation = 'relu')))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    #model.add(TimeDistributed(Dropout(0.25)))
                                      
    model.add(TimeDistributed(Flatten()))
                                      
    model.add(LSTM(32))
                                      
    model.add(Dense(10, activation = 'softmax'))

    ########################################################################################################################

    # Display the models summary.

    
    # Return the constructed LRCN model.
    return model

In [6]:
def create_convlstm_model():

    model = Sequential()
    model.add(ConvLSTM2D(filters = 4, kernel_size = (3, 3), activation = 'tanh',data_format = "channels_last",
                             recurrent_dropout=0.2, return_sequences=True, input_shape = (29,
                                                                                          100, 100, 3)))

    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last'))
    model.add(TimeDistributed(Dropout(0.2)))

    model.add(ConvLSTM2D(filters = 8, kernel_size = (3, 3), activation = 'tanh', data_format = "channels_last",
                         recurrent_dropout=0.2, return_sequences=True))

    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last'))
    model.add(TimeDistributed(Dropout(0.2)))

    model.add(ConvLSTM2D(filters = 14, kernel_size = (3, 3), activation = 'tanh', data_format = "channels_last",
                         recurrent_dropout=0.2, return_sequences=True))

    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last'))
    model.add(TimeDistributed(Dropout(0.2)))

    model.add(ConvLSTM2D(filters = 16, kernel_size = (3, 3), activation = 'tanh', data_format = "channels_last",
                         recurrent_dropout=0.2, return_sequences=True))

    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last'))
    #model.add(TimeDistributed(Dropout(0.2)))

    model.add(Flatten()) 

    model.add(Dense(10, activation = "softmax"))
    
    return model

In [7]:
dict_data = {'label': [], 'features': []}
counts = []
for skeleton_folder in tqdm_notebook(skeletons_action_folders):
    
    current_action = skeleton_folder.split('\\')[1]
    
    actors_folders = glob(skeleton_folder.replace('\\', '/')+'/*')
    
    for actor_folder in actors_folders:
        
        images = glob(actor_folder.replace('\\', '/')+'/*')
        
        if len(images) > 10:
        
            images.sort(key=len)

            dict_data['label'].append(current_action)

            w,h=Image.open(images[0].replace('\\', '/')).size
            N=len(images)

            # Create a numpy array of floats to store the average (assume RGB images)
            arr=np.zeros((h,w,3))

            feats = []

            # Build up average pixel intensities, casting each image as an array of floats
            for im in images[:29]:
                feats.append(np.array(Image.open(im)))

            # Round values in array and cast as 8-bit integer


            dict_data['features'].append(feats)
        counts.append(len(images))

  0%|          | 0/10 [00:00<?, ?it/s]

In [8]:
len(dict_data)

2

In [9]:
df = pd.DataFrame(dict_data)

In [10]:
len(df)

93

In [11]:
Counter(df['label'])

Counter({'bend': 9,
         'jack': 9,
         'jump': 9,
         'pjump': 9,
         'run': 10,
         'side': 9,
         'skip': 10,
         'walk': 10,
         'wave1': 9,
         'wave2': 9})

In [12]:
df['features'] = df['features'].apply(lambda x: np.asarray(x).astype(np.float32))

In [13]:
y = pd.get_dummies(df.label, prefix='')

In [14]:
y = y.apply(lambda x: np.asarray(x).astype(np.float32))

In [15]:
y.values[0]

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [271]:
features_train, features_test, labels_train, labels_test = train_test_split(df['features'].values, y.values,
                                                                            test_size = 0.25,
                                                                            random_state = 42)

In [272]:
features_train = np.asarray(list(features_train)).astype(np.float32)

In [273]:
features_test = np.asarray(list(features_test)).astype(np.float32)

In [274]:
features_train = features_train /255

In [275]:
features_test = features_test / 255

In [280]:
model = create_LRCN_model()
# model = create_convlstm_model()

In [281]:
# Create an Instance of Early Stopping Callback.
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 15, mode = 'min', restore_best_weights = True)
 
# Compile the model and specify loss function, optimizer and metrics to the model.
model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])

In [189]:
# Create an Instance of Early Stopping Callback
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 10, mode = 'min', restore_best_weights = True)

# Compile the model and specify loss function, optimizer and metrics values to the model
model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])

In [282]:
# Start training the model.
model.fit(x = features_train, y = labels_train, epochs = 70, batch_size = 8, validation_split = 0.2,callbacks = [early_stopping_callback])

Epoch 1/70
7/7 [==============================] - 7s 753ms/step - loss: 2.3522 - accuracy: 0.0545 - val_loss: 2.3315 - val_accuracy: 0.0000e+00
Epoch 2/70
7/7 [==============================] - 3s 508ms/step - loss: 2.2867 - accuracy: 0.1273 - val_loss: 2.3427 - val_accuracy: 0.0000e+00
Epoch 3/70
7/7 [==============================] - 4s 557ms/step - loss: 2.2755 - accuracy: 0.1091 - val_loss: 2.3437 - val_accuracy: 0.0714
Epoch 4/70
7/7 [==============================] - 4s 529ms/step - loss: 2.2643 - accuracy: 0.1818 - val_loss: 2.3530 - val_accuracy: 0.0714
Epoch 5/70
7/7 [==============================] - 4s 535ms/step - loss: 2.2427 - accuracy: 0.2182 - val_loss: 2.3404 - val_accuracy: 0.0714
Epoch 6/70
7/7 [==============================] - 4s 547ms/step - loss: 2.2039 - accuracy: 0.2545 - val_loss: 2.2950 - val_accuracy: 0.2143
Epoch 7/70
7/7 [==============================] - 3s 490ms/step - loss: 2.1286 - accuracy: 0.2545 - val_loss: 2.1230 - val_accuracy: 0.2857
Epoch 8/70
7

In [283]:
preds = model.predict(features_test)

In [284]:
y_pred = [pred.argmax() for pred in preds]

In [285]:
y_pred

[4, 2, 6, 4, 0, 2, 4, 7, 1, 4, 8, 3, 9, 6, 1, 6, 7, 3, 5, 7, 1, 6, 8, 3]

In [286]:
labels_test[8]

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [287]:
y_true = []
for vect in labels_test:
    for i, value in enumerate(vect):
        if value == 1:
            
            y_true.append(i)

In [288]:
y_true

[4, 2, 6, 7, 0, 2, 4, 7, 1, 4, 8, 3, 9, 6, 1, 0, 2, 3, 5, 7, 1, 7, 8, 3]

In [289]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.50      0.67         2
           1       1.00      1.00      1.00         3
           2       1.00      0.67      0.80         3
           3       1.00      1.00      1.00         3
           4       0.75      1.00      0.86         3
           5       1.00      1.00      1.00         1
           6       0.50      1.00      0.67         2
           7       0.67      0.50      0.57         4
           8       1.00      1.00      1.00         2
           9       1.00      1.00      1.00         1

    accuracy                           0.83        24
   macro avg       0.89      0.87      0.86        24
weighted avg       0.87      0.83      0.83        24



In [290]:
print(accuracy_score(y_true, y_pred))

0.8333333333333334


In [16]:
loo = LeaveOneOut()

In [93]:
y_true_all = pickle.load(open('y_true_all.pkl', 'rb'))
y_pred_all = pickle.load(open('y_pred_all.pkl', 'rb'))

In [13]:
len(y_true_all)

33

In [29]:
temp = []
checkpoint = 0

for train_index, test_index in tqdm_notebook(loo.split(df['features'].values)):
    if checkpoint >= 0:
        y = pd.get_dummies(df['label'], prefix='')

        y = y.apply(lambda x: np.asarray(x).astype(np.float32))
        labels_train, labels_test = y.values[train_index], y.values[test_index]
        y_true = []
        for vect in labels_test:
            for i, value in enumerate(vect):
                if value == 1:

                    y_true.append(i)
        temp.append(y_true[0])
    checkpoint += 1

0it [00:00, ?it/s]

In [19]:
print(y_true_all)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3]


In [18]:
print(y_pred_all)

[0, 0, 0, 0, 3, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 6, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 9]


In [30]:
print(temp)

[3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9]


In [28]:
y_pred_all = []
y_true_all = []
acertos = 0
checkpoint = 0
for train_index, test_index in tqdm_notebook(loo.split(df['features'].values)):
    print(checkpoint)
    if checkpoint >= 0:
        
        model = create_LRCN_model()

        # Create an Instance of Early Stopping Callback.
        early_stopping_callback = EarlyStopping(monitor = 'loss', patience = 15, mode = 'min', restore_best_weights = True)

        # Compile the model and specify loss function, optimizer and metrics to the model.
        model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])

        y = pd.get_dummies(df['label'], prefix='')

        y = y.apply(lambda x: np.asarray(x).astype(np.float32))

        features_train, features_test = df['features'].values[train_index], df['features'].values[test_index]
        labels_train, labels_test = y.values[train_index], y.values[test_index]

        features_train = np.asarray(list(features_train)).astype(np.float32)

        features_test = np.asarray(list(features_test)).astype(np.float32)

        features_train = features_train / 255

        features_test = features_test / 255

        # Start training the model.
        model.fit(x = features_train, y = labels_train, epochs = 70, batch_size = 16, callbacks=[early_stopping_callback])

        preds = model.predict(features_test)

        y_pred = [pred.argmax() for pred in preds]

        y_pred_all.append(y_pred[0])
        y_true = []
        for vect in labels_test:
            for i, value in enumerate(vect):
                if value == 1:

                    y_true.append(i)
        y_true_all.append(y_true[0])

        if y_pred[0] == y_true[0]:
            acertos += 1
        print(str(acertos)+'/'+str(len(y_pred_all)))
        
    checkpoint += 1

0it [00:00, ?it/s]

0
Epoch 1/70
6/6 [==============================] - 8s 1s/step - loss: 2.3479 - accuracy: 0.1196
Epoch 2/70
6/6 [==============================] - 6s 1s/step - loss: 2.2937 - accuracy: 0.1087
Epoch 3/70
6/6 [==============================] - 5s 811ms/step - loss: 2.2749 - accuracy: 0.0761
Epoch 4/70
6/6 [==============================] - 6s 1s/step - loss: 2.2319 - accuracy: 0.2391
Epoch 5/70
6/6 [==============================] - 7s 1s/step - loss: 2.1614 - accuracy: 0.3261
Epoch 6/70
6/6 [==============================] - 6s 1s/step - loss: 2.0237 - accuracy: 0.3478
Epoch 7/70
6/6 [==============================] - 7s 1s/step - loss: 1.8856 - accuracy: 0.3696
Epoch 8/70
6/6 [==============================] - 9s 1s/step - loss: 1.7551 - accuracy: 0.3587
Epoch 9/70
6/6 [==============================] - 9s 2s/step - loss: 1.9089 - accuracy: 0.2174
Epoch 10/70
6/6 [==============================] - 9s 2s/step - loss: 1.6255 - accuracy: 0.4022
Epoch 11/70
6/6 [===========================

6/6 [==============================] - 8s 1s/step - loss: 0.4160 - accuracy: 0.9130
Epoch 32/70
6/6 [==============================] - 9s 1s/step - loss: 0.4315 - accuracy: 0.8804
Epoch 33/70
6/6 [==============================] - 9s 2s/step - loss: 0.3998 - accuracy: 0.8913
Epoch 34/70
6/6 [==============================] - 8s 1s/step - loss: 0.3480 - accuracy: 0.9457
Epoch 35/70
6/6 [==============================] - 9s 1s/step - loss: 0.3991 - accuracy: 0.9022
Epoch 36/70
6/6 [==============================] - 7s 1s/step - loss: 0.3569 - accuracy: 0.9130
Epoch 37/70
6/6 [==============================] - 8s 1s/step - loss: 0.4131 - accuracy: 0.8587
Epoch 38/70
6/6 [==============================] - 8s 1s/step - loss: 0.3558 - accuracy: 0.9130
Epoch 39/70
6/6 [==============================] - 8s 1s/step - loss: 0.3390 - accuracy: 0.9457
Epoch 40/70
6/6 [==============================] - 9s 1s/step - loss: 0.3322 - accuracy: 0.9022
Epoch 41/70
6/6 [==============================] - 8

6/6 [==============================] - 6s 1s/step - loss: 0.0784 - accuracy: 1.0000
Epoch 61/70
6/6 [==============================] - 7s 1s/step - loss: 0.0817 - accuracy: 0.9891
Epoch 62/70
6/6 [==============================] - 6s 1s/step - loss: 0.0716 - accuracy: 0.9891
Epoch 63/70
6/6 [==============================] - 6s 1s/step - loss: 0.0722 - accuracy: 1.0000
Epoch 64/70
6/6 [==============================] - 6s 1s/step - loss: 0.0697 - accuracy: 1.0000
Epoch 65/70
6/6 [==============================] - 6s 1s/step - loss: 0.0521 - accuracy: 1.0000
Epoch 66/70
6/6 [==============================] - 6s 1s/step - loss: 0.0615 - accuracy: 0.9891
Epoch 67/70
6/6 [==============================] - 6s 1s/step - loss: 0.1073 - accuracy: 0.9783
Epoch 68/70
6/6 [==============================] - 6s 1s/step - loss: 0.1578 - accuracy: 0.9565
Epoch 69/70
6/6 [==============================] - 6s 1s/step - loss: 0.1287 - accuracy: 0.9674
Epoch 70/70
6/6 [==============================] - 6

6/6 [==============================] - 6s 954ms/step - loss: 0.8343 - accuracy: 0.7717
Epoch 20/70
6/6 [==============================] - 7s 1s/step - loss: 0.8131 - accuracy: 0.7609
Epoch 21/70
6/6 [==============================] - 7s 1s/step - loss: 0.6999 - accuracy: 0.7826
Epoch 22/70
6/6 [==============================] - 7s 1s/step - loss: 0.6485 - accuracy: 0.8043
Epoch 23/70
6/6 [==============================] - 7s 1s/step - loss: 0.7058 - accuracy: 0.7609
Epoch 24/70
6/6 [==============================] - 7s 1s/step - loss: 0.6805 - accuracy: 0.8370
Epoch 25/70
6/6 [==============================] - 7s 1s/step - loss: 0.5561 - accuracy: 0.8696
Epoch 26/70
6/6 [==============================] - 7s 1s/step - loss: 0.4997 - accuracy: 0.8804
Epoch 27/70
6/6 [==============================] - 7s 1s/step - loss: 0.5704 - accuracy: 0.8261
Epoch 28/70
6/6 [==============================] - 7s 1s/step - loss: 0.5636 - accuracy: 0.7935
Epoch 29/70
6/6 [==============================] 

...

6/6 [==============================] - 7s 1s/step - loss: 0.2990 - accuracy: 0.9674
Epoch 40/70
6/6 [==============================] - 7s 1s/step - loss: 0.3240 - accuracy: 0.9130
Epoch 41/70
6/6 [==============================] - 8s 1s/step - loss: 0.2834 - accuracy: 0.9674
Epoch 42/70
6/6 [==============================] - 7s 1s/step - loss: 0.2843 - accuracy: 0.9565
Epoch 43/70
6/6 [==============================] - 7s 1s/step - loss: 0.2712 - accuracy: 0.9348
Epoch 44/70
6/6 [==============================] - 7s 1s/step - loss: 0.2273 - accuracy: 0.9674
Epoch 45/70
6/6 [==============================] - 8s 1s/step - loss: 0.1836 - accuracy: 1.0000
Epoch 46/70
6/6 [==============================] - 7s 1s/step - loss: 0.1958 - accuracy: 0.9674
Epoch 47/70
6/6 [==============================] - 7s 1s/step - loss: 0.1754 - accuracy: 0.9891
Epoch 48/70
6/6 [==============================] - 7s 1s/step - loss: 0.1660 - accuracy: 0.9783
Epoch 49/70
6/6 [==============================] - 7

6/6 [==============================] - 10s 2s/step - loss: 0.0443 - accuracy: 1.0000
Epoch 70/70
6/6 [==============================] - 10s 2s/step - loss: 0.0413 - accuracy: 1.0000
66/85
85
Epoch 1/70
6/6 [==============================] - 12s 2s/step - loss: 2.3455 - accuracy: 0.0761
Epoch 2/70
6/6 [==============================] - 9s 2s/step - loss: 2.2984 - accuracy: 0.1196
Epoch 3/70
6/6 [==============================] - 9s 2s/step - loss: 2.2732 - accuracy: 0.1522
Epoch 4/70
6/6 [==============================] - 9s 2s/step - loss: 2.2519 - accuracy: 0.1739
Epoch 5/70
6/6 [==============================] - 8s 1s/step - loss: 2.2057 - accuracy: 0.4239
Epoch 6/70
6/6 [==============================] - 9s 1s/step - loss: 2.1032 - accuracy: 0.4022
Epoch 7/70
6/6 [==============================] - 9s 1s/step - loss: 1.9038 - accuracy: 0.4891
Epoch 8/70
6/6 [==============================] - 9s 1s/step - loss: 1.8360 - accuracy: 0.3804
Epoch 9/70
6/6 [==============================] 

6/6 [==============================] - 9s 2s/step - loss: 0.5244 - accuracy: 0.9130
Epoch 30/70
6/6 [==============================] - 7s 1s/step - loss: 0.5319 - accuracy: 0.9022
Epoch 31/70
6/6 [==============================] - 9s 1s/step - loss: 0.5008 - accuracy: 0.9022
Epoch 32/70
6/6 [==============================] - 9s 2s/step - loss: 0.4795 - accuracy: 0.9239
Epoch 33/70
6/6 [==============================] - 8s 1s/step - loss: 0.4177 - accuracy: 0.9239
Epoch 34/70
6/6 [==============================] - 9s 1s/step - loss: 0.3847 - accuracy: 0.9457
Epoch 35/70
6/6 [==============================] - 9s 1s/step - loss: 0.3800 - accuracy: 0.9457
Epoch 36/70
6/6 [==============================] - 9s 2s/step - loss: 0.3785 - accuracy: 0.9239
Epoch 37/70
6/6 [==============================] - 8s 1s/step - loss: 0.3833 - accuracy: 0.9239
Epoch 38/70
6/6 [==============================] - 9s 1s/step - loss: 0.3411 - accuracy: 0.9565
Epoch 39/70
6/6 [==============================] - 9

6/6 [==============================] - 9s 1s/step - loss: 0.1650 - accuracy: 0.9783
Epoch 60/70
6/6 [==============================] - 9s 1s/step - loss: 0.1559 - accuracy: 0.9674
Epoch 61/70
6/6 [==============================] - 9s 2s/step - loss: 0.1347 - accuracy: 1.0000
Epoch 62/70
6/6 [==============================] - 8s 1s/step - loss: 0.1374 - accuracy: 0.9891
Epoch 63/70
6/6 [==============================] - 9s 1s/step - loss: 0.1231 - accuracy: 0.9891
Epoch 64/70
6/6 [==============================] - 9s 1s/step - loss: 0.1105 - accuracy: 1.0000
Epoch 65/70
6/6 [==============================] - 9s 1s/step - loss: 0.1093 - accuracy: 1.0000
Epoch 66/70
6/6 [==============================] - 9s 1s/step - loss: 0.0966 - accuracy: 1.0000
Epoch 67/70
6/6 [==============================] - 8s 1s/step - loss: 0.0868 - accuracy: 1.0000
Epoch 68/70
6/6 [==============================] - 8s 1s/step - loss: 0.0880 - accuracy: 1.0000
Epoch 69/70
6/6 [==============================] - 8

6/6 [==============================] - 6s 1s/step - loss: 0.8369 - accuracy: 0.7065
Epoch 18/70
6/6 [==============================] - 7s 1s/step - loss: 0.7016 - accuracy: 0.8370
Epoch 19/70
6/6 [==============================] - 7s 1s/step - loss: 0.6367 - accuracy: 0.8370
Epoch 20/70
6/6 [==============================] - 6s 1s/step - loss: 0.6617 - accuracy: 0.8261
Epoch 21/70
6/6 [==============================] - 6s 1s/step - loss: 0.7314 - accuracy: 0.7826
Epoch 22/70
6/6 [==============================] - 7s 1s/step - loss: 0.8012 - accuracy: 0.7391
Epoch 23/70
6/6 [==============================] - 7s 1s/step - loss: 0.7113 - accuracy: 0.7826
Epoch 24/70
6/6 [==============================] - 6s 801ms/step - loss: 0.6479 - accuracy: 0.8696
Epoch 25/70
6/6 [==============================] - 7s 1s/step - loss: 0.5826 - accuracy: 0.8370
Epoch 26/70
6/6 [==============================] - 5s 807ms/step - loss: 0.4858 - accuracy: 0.9022
Epoch 27/70
6/6 [=============================

In [20]:
y_true_all

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2]

In [25]:
y_pred_all

[0, 0, 0, 0, 6, 3, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 6, 2, 6, 2]

In [108]:
y_true = []
for vect in labels_test:
    for i, value in enumerate(vect):
        if value == 1:

            y_true.append(i)

In [22]:
print(accuracy_score(y_true_all, y_pred_all))

0.8181818181818182


In [23]:
print(classification_report(y_true_all, y_pred_all))

              precision    recall  f1-score   support

           0       1.00      0.78      0.88         9
           1       1.00      1.00      1.00         9
           2       1.00      0.50      0.67         4
           3       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0

    accuracy                           0.82        22
   macro avg       0.60      0.46      0.51        22
weighted avg       1.00      0.82      0.89        22



In [26]:
pickle.dump(y_true_all, open('y_true_all_0-21_heat.pkl', 'wb'))
pickle.dump(y_pred_all, open('y_pred_all_0-21_heat.pkl', 'wb'))

In [18]:
y_true_1 = pickle.load(open('y_true_all.pkl', 'rb'))
y_pred_1 = pickle.load(open('y_pred_all.pkl', 'rb'))

In [19]:
y_true_2 = pickle.load(open('y_true_all_33-62.pkl', 'rb'))
y_pred_2 = pickle.load(open('y_pred_all_33-62.pkl', 'rb'))

In [20]:
y_true_1.extend(y_true_2)
y_true_1.extend(y_true_all)

In [21]:
y_pred_1.extend(y_pred_2)
y_pred_1.extend(y_pred_all)

In [22]:
print(len(y_pred_1))
print(len(y_true_1))

81
81


In [23]:
print(classification_report(y_true_1, y_pred_1))

              precision    recall  f1-score   support

           0       1.00      0.89      0.94         9
           1       1.00      1.00      1.00         9
           2       0.80      0.89      0.84         9
           3       0.62      0.83      0.71         6
           4       0.80      0.67      0.73         6
           5       0.86      0.86      0.86         7
           6       0.67      0.29      0.40         7
           7       0.69      0.90      0.78        10
           8       1.00      0.78      0.88         9
           9       0.82      1.00      0.90         9

    accuracy                           0.83        81
   macro avg       0.83      0.81      0.80        81
weighted avg       0.84      0.83      0.82        81



In [24]:
accuracy_score(y_true_1, y_pred_1)

0.8271604938271605